In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os 
import glob

In [ ]:
# Get list of bag files
# path = "/home/user/Downloads/new/"
path = "/home/user/Downloads/zed_compare/"
os.chdir(path)
file_lst = glob.glob('*.csv')

In [ ]:
file_lst

In [ ]:
df = []
for i, file in enumerate(file_lst):
    file_path = path + file
    df = pd.read_csv(file_path)
    f = 1000 / (df["imu_Timestamp[sec]"][1000] - df["imu_Timestamp[sec]"][0])
    t = np.linspace(0, df.shape[0]/f, df.shape[0])
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=t, y = df.iloc[:, 13], name="orX"), secondary_y=False)
    fig.add_trace(go.Scatter(x=t, y = df.iloc[:, 14], name="ory"), secondary_y=False)
    fig.add_trace(go.Scatter(x=t, y = df.iloc[:, 15], name="orZ"), secondary_y=False)
    fig.add_trace(go.Scatter(x=t, y = df.iloc[:, 22], name="temp"), secondary_y=True)
    fig.update_layout(title_text=file)
    fig.update_xaxes(title_text="Time [s]")
    fig.update_yaxes(title_text="Angle [deg]", secondary_y=False)
    fig.update_yaxes(title_text="Temperature [C]", secondary_y=True)
    fig.show()

In [ ]:
px.line(df[0].iloc[:,4:7])

In [ ]:
fig = px.line(df.iloc[:,7:10])
fig.show()

In [ ]:
def ang_from_vel(ang_vel):
    deg = np.cumsum(ang_vel) / f
    return deg

def remove_drift(ang_y_gyr):
    drift = np.zeros(len(ang_y_gyr))
    # Only remove drift if end position is also on flat ground
    if np.abs((ang_y_gyr[0] - ang_y_gyr[-1])) < 3:
        drift = np.linspace(ang_y_gyr[0], ang_y_gyr[-1], len(ang_y_gyr))
    ang_y_no_drift = ang_y_gyr - drift
    return ang_y_no_drift, drift

# Calculate pitch angle
ang_gyr = ang_from_vel(df.iloc[:, 7:10])

# # Remove drift (assumes start and end pitch angle are the same)
# ang_y_no_drift, drift = remove_drift(ang_y_gyr)

In [ ]:
px.line(ang_gyr)

In [ ]:
px.line(df.iloc[:,10:13])

In [ ]:
px.line(df.iloc[:,13:16])

In [ ]:
px.line(df.iloc[:,19:23])

In [ ]:
df_new = ang_gyr.copy()
df_new.columns = ["orGyrX[deg]", "orGyrY[deg]", "orGyrZ[deg]"]
df_new.index = df.index
df_new = df_new.join(df.iloc[:,13:16])
df_new.index = df["imu_Timestamp[sec]"]
df_new.head()

In [ ]:
px.line(df_new)